In [5]:
import random
import hgtk  # 한글 자모 분리를 위한 라이브러리 (pip install hgtk)
import sys

# 6개 음소로 이루어진 단어 리스트 및 힌트
word_list = [
    ("심장", "몸의 중요한 장기 중 하나"),
    ("안정", "편안한 상태"),
    ("안녕", "누군가를 만났을 때 또는 헤어질 때 하는 인사"),
    ("옹졸", "쪼잔한 사람"),
    ("행정", "ㅇㅇ업무"),
]

# AI가 랜덤으로 정답 단어 선택
word, hint = random.choice(word_list)
word_jamos = list(hgtk.text.decompose(word).replace("ᴥ", ""))  # 자음+모음 분리

attempts = 6  # 총 기회
hint_used = False  # 힌트 사용 여부

print("🎮 한국판 무한워들 (자음+모음 6개 게임)!")
print("⚠️ '힌트'를 입력하면 한 번만 단어의 힌트를 받을 수 있습니다.")
print("-" * 50)

while attempts > 0:
    user_input = input("단어 입력: ").strip()
    sys.stdout.flush()  # 🛠 즉시 출력되도록 설정

    # 힌트 요청 처리
    if user_input == "힌트":
        if not hint_used:
            print(f"💡 힌트: {hint}")
            sys.stdout.flush()  # 🛠 즉시 출력
            hint_used = True
        else:
            print("❌ 이미 힌트를 사용했습니다!")
            sys.stdout.flush()  # 🛠 즉시 출력
        continue

    # 입력 검증 (자음+모음 개수 6개만 허용)
    try:
        user_jamos = list(hgtk.text.decompose(user_input).replace("ᴥ", ""))  # 자음+모음 분리
    except:
        print("⚠️ 올바른 한글 단어를 입력해주세요!")
        sys.stdout.flush()  # 🛠 즉시 출력
        continue

    if len(user_jamos) != 6:
        print("⚠️ 정확히 자음+모음 6개로 이루어진 단어만 입력하세요!")
        sys.stdout.flush()  # 🛠 즉시 출력
        continue

    feedback = []

    for i in range(6):
        if user_jamos[i] == word_jamos[i]:  # 정확한 위치 (초록색)
            feedback.append("🟩")
        elif user_jamos[i] in word_jamos:  # 포함되지만 위치 다름 (노란색)
            feedback.append("🟨")
        else:  # 아예 존재하지 않음 (빨간색)
            feedback.append("🟥")

    print(" ".join(feedback))  # 피드백 출력
    sys.stdout.flush()  # 🛠 즉시 출력
    attempts -= 1

    # 정답 확인
    if user_jamos == word_jamos:
        print("🎉 정답입니다! 게임 종료.")
        sys.stdout.flush()  # 🛠 즉시 출력
        break
    elif attempts == 0:
        print(f"❌ 실패! 정답은 '{word}'였습니다.")
        sys.stdout.flush()  # 🛠 즉시 출력

print("게임 종료!")
sys.stdout.flush()  # 🛠 즉시 출력

🎮 한국판 무한워들 (자음+모음 6개 게임)!
⚠️ '힌트'를 입력하면 한 번만 단어의 힌트를 받을 수 있습니다.
--------------------------------------------------
🟩 🟥 🟥 🟥 🟥 🟨
🟥 🟥 🟩 🟩 🟥 🟨
🟩 🟥 🟥 🟩 🟥 🟨
🟩 🟥 🟥 🟩 🟥 🟨
🟩 🟩 🟩 🟩 🟩 🟩
🎉 정답입니다! 게임 종료.
게임 종료!


In [7]:
import os
import openai
import random
import hgtk  # 한글 자모 분리를 위한 라이브러리 (pip install hgtk)
from dotenv import load_dotenv
import sys

# OpenAI API 키 설정
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

openai.api_key = openai_api_key
def ask_gpt(prompt):
    """GPT-4o 모델을 사용하여 질문에 대한 답변을 생성하는 함수"""
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": """당신은 한국어 단어 맞히기 게임을 위한 AI입니다.  
            자음과 모음 개수를 합쳐서 정확히 6개인 한 단어를 선정하고, 이에 대한 간단한 힌트를 제공합니다.
             
             ### 단어 선정 예시 ###
            예시:
            단어: 안녕 / 힌트: 사람과 사람이 만났을 때 하거나 헤어질 때 하는 인사
            단어: 안정 / 힌트: 마음이나 상태가 편안하고 조용한 상태
            단어: 심장 / 힌트: 몸의 중요한 장기 중 하나
             
             ### 출력 형식 ###
            반드시 단어와 힌트만 '단어: xxx / 힌트: xxx' 형식으로 답변하세요."""},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

# AI에게 6개 음소로 구성된 단어 요청 (정확히 6개 음소인지 체크)
while True:
    ai_output = ask_gpt("자음+모음이 6개인 한국어 단어와 힌트를 하나 선정해줘.")
    try:
        word, hint = ai_output.split(" / ")
        word = word.replace("단어:", "").strip()
        hint = hint.replace("힌트:", "").strip()
        
        # 단어를 자음+모음 분리하여 길이 체크
        word_jamos = list(hgtk.text.decompose(word).replace("ᴥ", ""))  # 자음+모음 분리
        
        if len(word_jamos) == 6:
            break  # 정확한 길이면 게임 시작

    except:
        print("❌ AI가 올바른 형식으로 단어를 생성하지 않았습니다. 다시 시도하세요.")

print("🎮 GPT-4o 기반 단어 맞히기 게임! '종료'를 입력하면 게임이 끝납니다.")
print("=" * 100)
print(f"💡 AI가 단어를 선정했습니다! (힌트 사용 가능)")

attempts = 6  # 총 기회
hint_used = False  # 힌트 사용 여부

print("=" * 100)

while attempts > 0:
    user_input = input("단어 입력: ").strip()
    sys.stdout.flush()  # 즉시 출력

    if user_input == "종료":
        print("게임을 종료합니다.")
        break

    # 힌트 요청 처리
    if user_input == "힌트":
        if not hint_used:
            print(f"💡 힌트: {hint}")
            hint_used = True
        else:
            print("❌ 이미 힌트를 사용했습니다!")
        sys.stdout.flush()
        continue

    # 입력 검증 (자음+모음 개수 6개만 허용)
    try:
        user_jamos = list(hgtk.text.decompose(user_input).replace("ᴥ", ""))  # 자음+모음 분리
    except:
        print("⚠️ 올바른 한글 단어를 입력해주세요!")
        sys.stdout.flush()
        continue

    if len(user_jamos) != 6:
        print(f"⚠️ 입력한 단어는 {len(user_jamos)}개의 음소를 가지고 있습니다. 정확히 6개의 자음+모음을 가진 단어를 입력하세요!")
        sys.stdout.flush()
        continue

    feedback = []

    for i in range(6):
        if i < len(user_jamos) and i < len(word_jamos):  # 인덱스 범위 체크
            if user_jamos[i] == word_jamos[i]:  # 정확한 위치 (초록색)
                feedback.append("🟩")
            elif user_jamos[i] in word_jamos:  # 포함되지만 위치 다름 (노란색)
                feedback.append("🟨")
            else:  # 아예 존재하지 않음 (빨간색)
                feedback.append("🟥")

    # 입력한 단어의 자음/모음 분리된 형태도 같이 출력
    print("단어:", " ".join(user_jamos))
    print("결과:", " ".join(feedback))
    print("=" * 100)
    sys.stdout.flush()  # 즉시 출력

    attempts -= 1

    # 정답 확인
    if user_jamos == word_jamos:
        print("🎉 정답입니다! 게임 종료.")
        sys.stdout.flush()
        break
    elif attempts == 0:
        print(f"❌ 실패! 정답은 '{word}'였습니다.")
        sys.stdout.flush()

print("게임 종료! 감사합니다.")
sys.stdout.flush()


🎮 GPT-4o 기반 단어 맞히기 게임! '종료'를 입력하면 게임이 끝납니다.
💡 AI가 단어를 선정했습니다! (힌트 사용 가능)
단어: ㅎ ㅐ ㅇ ㅅ ㅓ ㅇ
결과: 🟥 🟥 🟨 🟥 🟥 🟨
💡 힌트: 글씨를 쓰거나 그릴 때 사용하는 필기도구
단어: ㅇ ㅕ ㄴ ㅍ ㅣ ㄹ
결과: 🟩 🟩 🟩 🟩 🟩 🟩
🎉 정답입니다! 게임 종료.
게임 종료! 감사합니다.
